Iris with PyTorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# 모델 정의
class IrisModel(nn.Module):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.fc1 = nn.Linear(4, 50) # 입력 4, 출력 50
        self.fc2 = nn.Linear(50, 30) # 입력 50, 출력 30
        self.fc3 = nn.Linear(30, 3) # 입력 30, 출력 3 (softmax로 변환)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return self.softmax(self.fc3(x))

In [3]:
# 데이터 로드
iris = datasets.load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# 데이터를 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# DataLoader 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [8]:
# 모델, 손실 함수, 옵티마이저 설정
model = IrisModel()
criterion = nn.CrossEntropyLoss() # sparse_categorical_crossentropy에 해당
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        # 순전파
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss : {loss.item():.4f}')

Epoch [1/100], Loss : 1.1250
Epoch [2/100], Loss : 1.0995
Epoch [3/100], Loss : 1.0740
Epoch [4/100], Loss : 1.1037
Epoch [5/100], Loss : 1.0469
Epoch [6/100], Loss : 1.1375
Epoch [7/100], Loss : 1.0529
Epoch [8/100], Loss : 1.0383
Epoch [9/100], Loss : 1.0998
Epoch [10/100], Loss : 1.0431
Epoch [11/100], Loss : 1.0647
Epoch [12/100], Loss : 1.1133
Epoch [13/100], Loss : 1.0827
Epoch [14/100], Loss : 0.9777
Epoch [15/100], Loss : 1.1347
Epoch [16/100], Loss : 0.9711
Epoch [17/100], Loss : 0.9171
Epoch [18/100], Loss : 0.9329
Epoch [19/100], Loss : 0.8801
Epoch [20/100], Loss : 1.0160
Epoch [21/100], Loss : 0.8704
Epoch [22/100], Loss : 0.8839
Epoch [23/100], Loss : 0.9940
Epoch [24/100], Loss : 1.0466
Epoch [25/100], Loss : 0.8185
Epoch [26/100], Loss : 0.7393
Epoch [27/100], Loss : 0.9419
Epoch [28/100], Loss : 0.8083
Epoch [29/100], Loss : 0.7992
Epoch [30/100], Loss : 0.7561
Epoch [31/100], Loss : 0.8563
Epoch [32/100], Loss : 0.7307
Epoch [33/100], Loss : 0.9082
Epoch [34/100], Los

In [9]:
# 평가
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor).float().mean()
    print(f"Test Accuracy : {accuracy.item() * 100:.2f}%")

Test Accuracy : 97.78%
